# Interval Around the Current Price
This strategy will provide liquidity around the current price in a fixed interval. We will rebalance our position every update interval as discussed in the beginning of chapter 4. Let pc be the current price and a ∈ [10, 40000], then we will set our position to [pc − a, pc + a] at every update interval.
We will fill the interval as much as possible. However, it is usually not possible to use up both tokens, so the remaining token will be left outside the pool.

In [1]:
from datetime import timedelta, date
from typing import Dict
from _decimal import Decimal
import pandas as pd

from demeter import TokenInfo, Actuator, ChainType, MarketInfo, Strategy, EvaluatorEnum
from demeter.uniswap import UniV3Pool, UniLpMarket

In [2]:
pd.options.display.max_columns = None
pd.set_option("display.width", 5000)

In [3]:
class IntervalsAroundtheCurrentPrice(Strategy):
    def __init__(self, a=10, b=1, update_interval=timedelta(days=1)):
        super().__init__()
        self.a = a
        self.b = b

    def initialize(self):
        lp_market: UniLpMarket = self.broker.markets[market_key]
        init_price = lp_market.market_status.data.price

        lp_market.add_liquidity(init_price - self.a, init_price + self.a)

In [4]:
usdc = TokenInfo(name="usdc", decimal=6)  # declare  token0
eth = TokenInfo(name="eth", decimal=18)  # declare token1
pool = UniV3Pool(usdc, eth, 0.05, usdc)  # declare pool
market_key = MarketInfo("market1")

actuator = Actuator()  # declare actuator
broker = actuator.broker
market = UniLpMarket(market_key, pool)

broker.add_market(market)
broker.set_balance(usdc, 2000)
broker.set_balance(eth, 0)

actuator.strategy = IntervalsAroundtheCurrentPrice(400, 200)

market.data_path = "../data"
market.load_data(ChainType.polygon.name, "0x45dda9cb7c25131df268515131f647d726f50608", date(2023, 8, 13), date(2023, 8, 17))
actuator.set_price(market.get_price_from_data())
# actuator.run()

2023-11-03 10:14:00,381 - INFO - start load files from 2023-08-13 to 2023-08-17...
2023-11-03 10:14:00,440 - INFO - load file complete, preparing...
2023-11-03 10:14:00,740 - INFO - data has been prepared


In [5]:
actuator.run(evaluator=[
        EvaluatorEnum.max_draw_down,
        EvaluatorEnum.annualized_returns,
        EvaluatorEnum.net_value,
        EvaluatorEnum.profit,
        EvaluatorEnum.net_value_up_down_rate,
        EvaluatorEnum.eth_up_down_rate,
        EvaluatorEnum.position_fee_profit,
        EvaluatorEnum.position_fee_annualized_returns,
        EvaluatorEnum.position_market_time_rate,
    ])
evaluating_result: Dict[EvaluatorEnum, Decimal] = actuator.evaluating_indicator

actuator.save_result(
    path="./result",  # save path
    account=True,  # save account status list as a csv file
    actions=True,  # save actions as a json file and a pickle file
)

2023-11-03 10:14:00,766 - INFO - init strategy...
2023-11-03 10:14:00,775 - INFO - start main loop...
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████| 7200/7200 [00:04<00:00, 1612.50it/s]
2023-11-03 10:14:05,278 - INFO - main loop finished
2023-11-03 10:14:05,337 - INFO - Start calculate evaluating indicator...
/Users/florije/PycharmProjects/demeter/demeter/core/evaluating_indicator.py:85: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fee_df.sort_values(by=["market1_base_uncollected", "market1_quote_uncollected"], ascending=[False, False], inplace=True)
2023-11-03 10:14:05,376 - INFO - Evaluating indicator has finished it's job.
2023-11-03 10:14:05,377 - INFO - Print actuator summary
2023-11-03 10:14:05,388 - INFO - Backtesting finished, 

Final account status                              
Token balance in broker       
USDC      :2000                     ETH       :0                        
Position value in markets     
market1(UniLpMarket)
token0    :USDC                     token1    :ETH                      fee       :0.0500                   is 0 base :True                     
positions 
   lower_tick  upper_tick pending0 pending1  liquidity
0      199141      203540    0E-10    0E-22          0

Account balance history                           
                                         net_value  USDC ETH market1_net_value market1_base_uncollected market1_quote_uncollected market1_base_in_position market1_quote_in_position  market1_position_count
2023-08-13 00:00:00  2000.000000000000000000000000  2000   0             0E-46                   0.0000                     0E-22                        0                         0                       1
2023-08-13 00:01:00  2000.000000000000000000000000  2000   0     

['./result/backtest-20231103-101405.account.csv',
 './result/backtest-20231103-101405.action.json',
 './result/backtest-20231103-101405.action.pkl']